In [1]:
import numpy as np
import pandas as pd
from sunpy.physics.differential_rotation import rot_hpc
from astropy import units as u
from shapely.geometry import Polygon, Point
from shapely import wkt
import os
import datetime
from sunpy.time import parse_time

In [2]:
def unicode2polygon(bbox_array):
    if isinstance(bbox_array, basestring):
        bbox_array = wkt.loads(bbox_array)
    else:
        bbox_array = map(lambda x: wkt.loads(x), bbox_array)
#         for i, elem in bbox_array:
#             bbox_array[i] = wkt.loads(elem)
    return bbox_array

In [7]:
def pull_sample_flare_1(inputFile, begin_time, end_time, max_dist_frm_center, min_peak_flux):
    flare_set = pd.read_csv(inputFile, delimiter = ',', header = 0)
    flare_set['event_starttime'] = map(parse_time, flare_set['event_starttime'])
    flare_set['event_endtime'] = map(parse_time, flare_set['event_endtime'])
    flare_set = flare_set.ix[flare_set['event_starttime']>=begin_time]
    flare_set = flare_set.ix[flare_set['event_endtime']<=end_time]     
    flare_set = flare_set.ix[(flare_set['dist_frm_center']/966)<=(0.01*max_dist_frm_center)]
    flare_set = flare_set.ix[flare_set['sum_peakflux']>=min_peak_flux]
    return flare_set

In [4]:
def pull_sample_ef(inputFile, begin_time, end_time, delta_t, min_max_b_strength):
    ef_set = pd.read_csv(inputFile, delimiter = ',', header = 0)
    ef_set['event_starttime'] = map(parse_time, ef_set['event_starttime'])
    ef_set['event_endtime'] = map(parse_time, ef_set['event_endtime'])
#     ef_set = ef_set.ix[ef_set['event_probability']==1]
    ef_set = ef_set.ix[ef_set['maxmagfieldstrength']>=100]
    ef_set = ef_set.ix[ef_set['event_starttime']>=(begin_time-delta_t)]
    ef_set = ef_set.ix[ef_set['event_endtime']<=(end_time-delta_t)]
    ef_set = ef_set.rename(columns={'event_starttime': 'ef_starttime', 'event_endtime': 'ef_endtime',
                                    'hpc_bbox': 'ef_hpc_bbox', 'hpc_coord': 'ef_hpc_coord',
                                    'hpc_radius': 'ef_hpc_radius','hpc_x' : 'ef_hpc_x','hpc_y': 'ef_hpc_y', 
                                    'SOL_standard' : 'ef_SOL_standard'})
    return ef_set

In [5]:
def associate_ef2(inputFile_fl, inputFile_ef, begin_year, begin_month, begin_day, end_year, end_month, end_day, 
                 max_dist_frm_center, min_peak_flux, min_max_b_strength, temporal_sep_hr, spatial_sep, 
                  output2file=False, out_file = None):
    #covert times to datetime objects
    delta_t = datetime.timedelta(hours = temporal_sep_hr)
    begin_time = datetime.datetime(begin_year, begin_month, begin_day)
    end_time = datetime.datetime(end_year, end_month, end_day) 
    
    #pull appropiate flare events given input parameters
    flare_set = pull_sample_flare_1(inputFile_fl, begin_time, end_time, max_dist_frm_center, min_peak_flux)
    #pull potentially related ef events given input parameters
    ef_set = pull_sample_ef(inputFile_ef, begin_time, end_time, delta_t, min_max_b_strength)
    #set the radius of the sun in arc secs
    r = 966
    #how many flare events working with after cull by intial spatial, temporal, and size paramters
    length = flare_set.shape[0]
    #list of zeroes with length of number of flare events
    zeroes = [0 for i in range(length)]
    float_zeroes = [0.0 for i in range(length)]
    nones = [None for i in range(length)]
    #create columns (filled with zeroes) for tracking associated events and number of associated events
    flare_set.loc[:, 'associated_ef'] = nones
    flare_set.loc[:, 'num_associated_ef'] = zeroes
    flare_set.loc[:, 'ef_association_strength'] = float_zeroes
    #get a list of ef keywords of relevance to flare events
    ef_keywords = list(np.genfromtxt('keywords_ef_append_fl.csv', delimiter=',', dtype=str))
    #create columns filled with zeroes for each ef keyword
    for elem in ef_keywords:
        flare_set.loc[:, elem] = nones
    #create an empty list to fill with row indexes of duplicated flare events to delete at end of function
    to_be_deleted = []
    #get indexes in list 
    idx = flare_set['event_starttime'].index.tolist()
    #last row index
    idx_last = idx[-1]
    #convert start, peak, and end times to datetime objects
    flare_set['event_starttime'] = map(parse_time, flare_set['event_starttime'])
    flare_set['event_peaktime'] = map(parse_time, flare_set['event_peaktime'])
    flare_set['event_endtime'] = map(parse_time, flare_set['event_endtime'])
    #set positional row index 
    i = -1
    for elem in flare_set['event_starttime']:
        i += 1
        #print which flare event function is currently processing, so the user has an idea of how much longer
        #program will need to run
        if (i+1)%10 == 0:
            print '%d / %d events' %((i+1), length)
        #calculate what time the earliest associated ef could have occured
        start_ef = elem-delta_t
        #begin eliminating ef events based on temporal parameters—ef events must end after the earliest possible 
        #event start time and end before the flare event begins
        ef_search = ef_set.ix[ef_set['ef_endtime']>=start_ef]
        ef_search = ef_search.ix[ef_search['ef_endtime']<=elem]
        #create a counter for the number of associated ef events for a particular flare
        num_associated_ef = 0
        #as long as the temporal search does not eliminate all possible related ef events, proceed
        if ef_search.empty == False:
            #create a shapely point object from the flare's mean coordinates
            #keep in mind, Point in HPC units 
            fl_point = Point((flare_set['hpc_x'].values[i], flare_set['hpc_y'].values[i]))
            #run this loop if temporal qualifications have limited ef_search to only one ef event
            if ef_search.shape[0] == 1:
                #create a shapely polygon object from ef's location
                ef_poly = wkt.loads(ef_search['ef_hpc_bbox'].values[0])
                #calculate the minimum 2D distance between the ef's polygon and the flare's mean coordinate
                chord = fl_point.distance(ef_poly)
                #calculate the minimum 3D distance along the sun's curved surface between the ef & flare events
                #assumes the same radius for all events
                s = r*np.arcsin(chord/(2*r))
                #determine whether the spatial distance between ef and flare meets the set parameter
                if s <= spatial_sep:
                    #have found an associated ef
                    one_associated_ef(i, 0, flare_set, ef_search, s, ef_keywords, idx)
            #run this loop if temporal qualifications have limited ef_search to a list of possible ef
            #works in the same manner as the loop for a single event except each element of the list is
            #run through consecutively  
            else:
                ef_search['ef_hpc_bbox'] = unicode2polygon(ef_search['ef_hpc_bbox'])
                #create a dummy counter variable because python won't let me use enumerate on a dataframe column
                j = 0
                #create an empty list to add found event indices to for later access
                listOindices = []
                listOs = []
                for elem in ef_search['ef_hpc_bbox'].values:
                    chord = fl_point.distance(elem)
                    s = r*np.arcsin(chord/(2*r))
                    if s <= spatial_sep:
                        num_associated_ef+=1 
                        listOindices.append(j)
                        listOs.append(s)
                    j+=1
                #run if we've found at least one associted ef event
                if num_associated_ef == 1:
                    one_associated_ef(i, listOindices[0], flare_set, ef_search, s, ef_keywords, idx)
                    flare_set['num_associated_ef'].values[i] = num_associated_ef
                elif num_associated_ef > 1:
                    flare_set['num_associated_ef'].values[i] = num_associated_ef
                    highest_association = 0
                    highest_association_index = 0
                    for counter, index in enumerate(listOindices):
                        if listOs[counter]==0: listOs[counter]=1
                        temporal_sep = pd.to_timedelta(flare_set['event_peaktime'].values[i]-ef_search['ef_endtime'].values[index])
                        temporal_sep = (temporal_sep.total_seconds())/60
                        ef_association_strength = (((ef_search['maxmagfieldstrength'].values[index])**2)/
                                                   (listOs[counter]*temporal_sep))
                        if ef_association_strength >= highest_association:
                            highest_association_index = index
                            highest_association_strength = ef_association_strength
                    flare_set['associated_ef'].values[i] = ef_search['ef_SOL_standard'].values[highest_association_index]
                    for elem in ef_keywords:
                        flare_set.loc[idx[i], elem] = ef_search[elem].values[highest_association_index]
                    flare_set['ef_association_strength'].values[i] = highest_association_strength
                    
    #create boolean var to easily determine whether flare associated with an ef
    k = 0
    is_ef = [0 for i in range(flare_set.shape[0])]
    for elem in flare_set['associated_ef']:
        if elem!=None:
            is_ef[k] = 1
        k+=1
    flare_set.loc[:, 'is_ef'] = is_ef
    
    #write dataframe to a csv file depending on initial parameters
    if output2file == True:
        if out_file == None:
            #create a generic name for file based on search parameters if no file name specified
             out_file = ('flare_search_'+str(begin_time)[0:10]+'_'+ str(end_time)[0:10]+'_'+str(temporal_sep_hr)+'_'+
                         str(max_dist_frm_center)+'_'+str(min_peak_flux)+'_'+str(min_max_b_strength)+'.csv')
        #import which keywords to keep for outported data
        flare_keywords = list(np.genfromtxt('keywords_flare_with_goes_with_ar_with_sg_with_er.csv', delimiter=',', 
                                            dtype=str))
        #add to these keywords descriptors of associated ef
        flare_keywords.extend(['is_ef','associated_ef', 'num_associated_ef', 'ef_association_strength'])
        flare_keywords.extend(ef_keywords)
        #write to csv
        flare_set.to_csv(path_or_buf=out_file, columns = flare_keywords, index = False)
        
    return flare_set 

In [6]:
def one_associated_ef(flare_index, ef_index, fl_set, ef_set, s, ef_keywords, idx):
    #enter into the flare dataset information about its associated ef
    fl_set['associated_ef'].values[flare_index] = ef_set['ef_SOL_standard'].values[ef_index]
    for elem in ef_keywords:
        fl_set.loc[idx[flare_index], elem] = ef_set[elem].values[ef_index]
    #set the number of associated ef events to 1
    fl_set['num_associated_ef'].values[flare_index] = 1 
    #calculate the association strength of the ef to the flare
    #if flare is contained in ef, set the seperation distance to 1 so as to not divide by 0
    if s==0: s=1
    temporal_sep = pd.to_timedelta(fl_set['event_peaktime'].values[flare_index]-ef_set['ef_endtime'].values[ef_index])
    temporal_sep = (temporal_sep.total_seconds())/60
    fl_set['ef_association_strength'].values[flare_index] = (((ef_set['maxmagfieldstrength'].values[ef_index])**2)
                                                                /(s*temporal_sep))
    b2 = (ef_set['maxmagfieldstrength'].values[ef_index])**2


In [10]:
#associate_ef2(inputFile_fl, inputFile_ef, begin_year, begin_month, begin_day, end_year, end_month, end_day, 
#                  max_dist_frm_center, min_peak_flux, min_max_b_strength, temporal_sep_hr, spatial_sep, 
#                   output2file=False, out_file = None)

associate_ef2('flare_dataset_cleaned_30min_100arcsec_with_GOES_with_ar_with_sigmoid_with_er.csv', 'raw_ef.csv', 2010, 
              6, 1, 2016, 6, 1, 60, 250, 50, 12, 100, output2file=True, out_file = 'flare_dataset_ef2.csv')

10 / 6197 events
20 / 6197 events
30 / 6197 events
40 / 6197 events
50 / 6197 events
60 / 6197 events
70 / 6197 events
80 / 6197 events
90 / 6197 events
100 / 6197 events
110 / 6197 events
120 / 6197 events
130 / 6197 events
140 / 6197 events
150 / 6197 events
160 / 6197 events
170 / 6197 events
180 / 6197 events
190 / 6197 events
200 / 6197 events
210 / 6197 events
220 / 6197 events
230 / 6197 events
240 / 6197 events
250 / 6197 events
260 / 6197 events
270 / 6197 events
280 / 6197 events
290 / 6197 events
300 / 6197 events
310 / 6197 events
320 / 6197 events
330 / 6197 events
340 / 6197 events
350 / 6197 events
360 / 6197 events
370 / 6197 events
380 / 6197 events
390 / 6197 events
400 / 6197 events
410 / 6197 events
420 / 6197 events
430 / 6197 events
440 / 6197 events
450 / 6197 events
460 / 6197 events
470 / 6197 events
480 / 6197 events
490 / 6197 events
500 / 6197 events
510 / 6197 events
520 / 6197 events
530 / 6197 events
540 / 6197 events
550 / 6197 events
560 / 6197 events
5

,SOL_standard,event_starttime,event_endtime,event_peaktime,hpc_bbox,hpc_coord,hpc_radius,hpc_x,hpc_y,is_associated_fl,...,ef_axisorientationunit,ef_aspectratio,ef_proximityratio,ef_hpc_bbox,ef_hpc_coord,ef_hpc_radius,ef_hpc_x,ef_hpc_y,event_probability,is_ef
14,SOL2010-06-27T03:11:02L203C114,2010-06-27 03:11:02,2010-06-27 08:47:14,2010-06-27 03:15:26,"POLYGON((-230.4 -460.8,-153.6 -460.8,-153.6 -3...",POINT(-192 -422.4),463.988965,-192.0,-422.4,1,...,None,None,None,None,None,None,None,None,None,0
26,SOL2010-07-13T10:09:51L332C070,2010-07-13 10:09:51,2010-07-13 10:15:39,2010-07-13 10:12:15,"POLYGON((-460.8 230.4,-384 230.4,-384 307.2,-4...",POINT(-422.4 268.8),500.674745,-422.4,268.8,0,...,None,None,None,None,None,None,None,None,None,0
27,SOL2010-07-13T10:46:03L337C070,2010-07-13 10:46:03,2010-07-13 10:53:39,2010-07-13 10:48:39,"POLYGON((-460.8 230.4,-307.2 230.4,-307.2 307....",POINT(-345.6 268.8),437.827363,-345.6,268.8,1,...,None,None,None,None,None,None,None,None,None,0
28,SOL2010-07-15T01:35:15L331C070,2010-07-15 01:35:15,2010-07-15 01:38:03,2010-07-15 01:35:51,"POLYGON((-153.6 230.4,-76.8 230.4,-76.8 307.2,...",POINT(-115.2 268.8),292.445687,-115.2,268.8,1,...,None,None,None,None,None,None,None,None,None,0
29,SOL2010-07-17T00:19:15L336C070,2010-07-17 00:19:15,2010-07-17 00:22:27,2010-07-17 00:20:39,"POLYGON((307.2 230.4,384 230.4,384 307.2,307.2...",POINT(345.6 268.8),437.827363,345.6,268.8,1,...,None,None,None,None,None,None,None,None,None,0
37,SOL2010-07-25T11:49:25L208C111,2010-07-25 11:49:25,2010-07-25 11:57:01,2010-07-25 11:53:25,"POLYGON((76.8 -460.8,153.6 -460.8,153.6 -384,7...",POINT(115.2 -422.4),437.827363,115.2,-422.4,0,...,None,None,None,None,None,None,None,None,None,0
38,SOL2010-07-25T19:28:37L209C111,2010-07-25 19:28:37,2010-07-25 19:31:37,2010-07-25 19:28:49,"POLYGON((153.6 -460.8,230.4 -460.8,230.4 -384,...",POINT(192 -422.4),463.988965,192.0,-422.4,0,...,None,None,None,None,None,None,None,None,None,0
39,SOL2010-07-26T00:33:49L206C111,2010-07-26 00:33:49,2010-07-26 00:37:25,2010-07-26 00:34:49,"POLYGON((153.6 -460.8,230.4 -460.8,230.4 -384,...",POINT(192 -422.4),463.988965,192.0,-422.4,1,...,None,None,None,None,None,None,None,None,None,0
40,SOL2010-07-26T16:27:01L202C111,2010-07-26 16:27:01,2010-07-26 16:34:25,2010-07-26 16:28:49,"POLYGON((230.4 -460.8,307.2 -460.8,307.2 -384,...",POINT(268.8 -422.4),500.674745,268.8,-422.4,1,...,None,None,None,None,None,None,None,None,None,0
41,SOL2010-07-27T01:09:37L203C111,2010-07-27 01:09:37,2010-07-27 01:13:01,2010-07-27 01:11:13,"POLYGON((307.2 -460.8,384 -460.8,384 -384,307....",POINT(345.6 -422.4),545.766543,345.6,-422.4,1,...,None,None,None,None,None,None,None,None,None,0
